## Exercise 2: Compute rotational energy levels of water molecule
using the following geometrical parameters: O--H distances of 0.96 Angstrom and H--O--H valence bond angle of 104$^\circ$

In [49]:
import numpy as np


r1 = 0.96
r2 = 0.96
theta = 104.0 * np.pi / 180.0 # angle in radians
masses = [15.9994, 1.00797, 1.00797] # masses of O, H1, and H2 atoms

# Cartesian coordinates of atoms
xyz = np.array( [[0, 0, 0], \
                 [r1 * np.sin(theta / 2.0), 0, r1 * np.cos(theta / 2.0)], \
                 [-r2 * np.sin(theta / 2.0), 0, r2 * np.cos(theta / 2.0)]], dtype=np.float64)

# move origin to COM
com = np.dot(masses, xyz) / np.sum(masses)
xyz = xyz - com[np.newaxis, :]

# compute inertia tensor
delta = lambda a, b: 1 if a==b else 0
imom = np.zeros((3,3), dtype=np.float64)
for i in range(3):
    for j in range(3):
        imom[i,j] = -np.sum(xyz[:,i] * xyz[:,j] * masses[:]) \
                  + np.sum(np.sum(xyz[:,:]**2, axis=1) * masses[:]) * delta(i,j)

# diagonalize inertia tensor and compute rotational constants
diag, vec = np.linalg.eigh(imom)
A, B, C = 1/diag
print(A, B, C) # rotational constants in units 1/(atomic_mass*A^2)

# convert rotational constants such that T = A*J_a^2 + B*J_b^2 + C*J_c^2
planck = 6.62606896e-27           # Plank constant in erg a second
avogno = 6.0221415e+23            # Avogadro constant
vellgt = 2.99792458e+10           # Speed of light constant in centimetres per second
convert_to_cm = planck * avogno * 1e+16 / (8.0 * np.pi * np.pi * vellgt)
print(convert_to_cm)
A, B, C = 1/diag * convert_to_cm
print(A, B, C)

1.5989494757174432 0.8667934951373716 0.5620857571074049
16.857628229754965
26.954495819806272 14.612082492995729 9.475432726556981


In [50]:
# generate basis set indices
Jmax = 3

# J+, J-, Jz, and J^2 operators
Jplus = lambda J, k, c=1: (J, k-1, np.sqrt(J * (J + 1) - k * (k - 1)) * c if abs(k-1) <= J else 0)
Jminus = lambda J, k, c=1: (J, k+1, np.sqrt(J * (J + 1) - k * (k + 1)) * c if abs(k+1) <= J else 0)
Jz = lambda J, k, c=1: (J, k, k * c)
Jsq = lambda J, k, c=1: (J, k, J * (J + 1) * c)

# overlap integral
overlap = lambda J1, k1, c1, J2, k2, c2: c1 * c2 * delta(J1, J2) * delta(k1, k2)

In [68]:
for J in range(Jmax+1):
    k_list = [k for k in range(-J, J+1)]

    Jplus_mat = np.array([[overlap(*(J, k1, 1), *Jplus(*Jplus(J, k2))) for k1 in k_list] for k2 in k_list], dtype=np.float64)
    Jminus_mat = np.array([[overlap(*(J, k1, 1), *Jminus(*Jminus(J, k2))) for k1 in k_list] for k2 in k_list], dtype=np.float64)
    Jz_mat = np.array([[overlap(*(J, k1, 1), *Jz(*Jz(J, k2))) for k1 in k_list] for k2 in k_list], dtype=np.float64)
    Jsq_mat = np.array([[overlap(*(J, k1, 1), *Jsq(J, k2)) for k1 in k_list] for k2 in k_list], dtype=np.float64)
    
    hmat_a = (Jplus_mat + Jminus_mat) * (B-C)/4.0 + Jz_mat * (2*A - B - C)/2.0 + (B + C)/2.0 * Jsq_mat
    enr_a, vec_a = np.linalg.eigh(hmat_a)

    hmat_c = (Jplus_mat + Jminus_mat) * (A - B)/4.0 + Jz_mat * (2*C - A - B)/2.0 + (A + B)/2.0 * Jsq_mat
    enr_c, vec_c = np.linalg.eigh(hmat_c)

    for istate in range(len(k_list)):
        ind_a = np.argmax(vec_a[:,istate]**2)
        ind_c = np.argmax(vec_c[:,istate]**2)
        print(J, istate, enr_a[istate], enr_c[istate], k_list[ind_a], k_list[ind_c])


0 0 0.0 0.0 0 0
1 0 24.087515219552706 24.087515219552706 0 1
1 1 36.42992854636325 36.42992854636325 1 -1
1 2 41.566578312801994 41.566578312801994 -1 0
2 0 70.9636755455999 70.96367554559993 0 2
2 1 79.46830921902992 79.46830921902993 -1 -2
2 2 94.87825851834613 94.87825851834617 1 -1
2 3 131.90549849877777 131.90549849877777 2 1
2 4 133.20436861183592 133.204368611836 -2 0
3 0 138.4983039270043 138.49830392700432 0 -3
3 1 143.29526684217768 143.2952668421778 1 3
3 2 173.90183223086714 173.90183223086723 -1 -2
3 3 204.16804415743601 204.16804415743587 -2 2
3 4 210.19483154774792 210.19483154774787 2 1
3 5 279.45234859343753 279.45234859343765 -3 -1
3 6 279.66568180338066 279.66568180338083 3 0


In [71]:
from richmol.watie import RigidMolecule, SymtopBasis, Jxx, Jyy, Jzz, settings

water = RigidMolecule()
water.XYZ = ("angstrom", \
             "O", 0, 0, 0, \
             "H", r1*np.sin(theta/2.0), 0, r1*np.cos(theta/2.0), \
             "H", -r2*np.sin(theta/2.0), 0, r2*np.cos(theta/2.0) )

water.frame = "pas"
Bx, By, Bz = water.B
print("Bx, By, Bz rotational constants in cm^-1:", Bx, By, Bz)

Jmax = 3
enr_all = []
for J in range(Jmax+1):
    bas = SymtopBasis(J)
    H = Bx * Jxx(bas) + By * Jyy(bas) + Bz * Jzz(bas)
    hmat = bas.overlap(H)
    enr, vec = np.linalg.eigh(hmat.real)
    enr_all += [e for e in enr]
    bas2 = bas.rotate(krot=(vec.T, enr))   # rotate basis to the eigenvector representation
                                           #   and assign energies to new basis states
    # print states energies and assignments
    settings.assign_nprim = 1 # number of primitive states used for assignment (typically = 1)
    nprim = settings.assign_nprim
    for istate in range(bas2.nstates):
        print( J, bas2.sym[istate], " %12.6f"%bas2.enr[istate], \
                "  ".join(s+"=%s"%q for s,q, in zip(("| J","k","tau","abs(c)^2")*nprim, bas2.assign[istate])) )


Bx, By, Bz rotational constants in cm^-1: 26.958784982583502 14.614184327525283 9.476846602963152
0 A      0.000000 | J=0  k=0  tau=0  abs(c)^2= 1.0000
1 A     24.091031 | J=1  k=1  tau=1  abs(c)^2= 1.0000
1 A     36.435632 | J=1  k=1  tau=0  abs(c)^2= 1.0000
1 A     41.572969 | J=1  k=0  tau=1  abs(c)^2= 1.0000
2 A     70.974093 | J=2  k=2  tau=0  abs(c)^2= 0.8634
2 A     79.480356 | J=2  k=2  tau=1  abs(c)^2= 1.0000
2 A     94.892369 | J=2  k=1  tau=0  abs(c)^2= 1.0000
2 A    131.926171 | J=2  k=1  tau=1  abs(c)^2= 1.0000
2 A    133.225170 | J=2  k=0  tau=0  abs(c)^2= 0.8634
3 A    138.518771 | J=3  k=3  tau=1  abs(c)^2= 0.8726
3 A    143.316754 | J=3  k=3  tau=0  abs(c)^2= 0.9682
3 A    173.927433 | J=3  k=2  tau=1  abs(c)^2= 0.7138
3 A    204.199264 | J=3  k=2  tau=0  abs(c)^2= 1.0000
3 A    210.226679 | J=3  k=1  tau=1  abs(c)^2= 0.8726
3 A    279.496300 | J=3  k=1  tau=0  abs(c)^2= 0.9682
3 A    279.709646 | J=3  k=0  tau=1  abs(c)^2= 0.7138
